In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy as sp
import sklearn
import feature_engine as fe

In [10]:
scores = pd.read_csv("../dataset/scoring_data/diabetes.csv", index_col="PatientID")

In [11]:
scores

,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Outcome
PatientID,,,,,,,
18022,129,80,0,0,31.2,0.703,0
18024,159,64,0,0,27.4,0.294,0
18051,137,61,0,0,24.2,0.151,0
18114,113,50,10,85,29.5,0.626,0
18147,105,90,0,0,29.6,0.197,0
...,...,...,...,...,...,...,...
19906,78,50,32,88,31.0,0.248,1
19913,189,104,25,0,34.3,0.435,1
19914,90,68,42,0,38.2,0.503,1


In [12]:
"""Convert the given string to snake_case, needs better handling of edge cases"""

def normalize(text: str):

    if text.isupper():
        print(text)
        return text.lower()
    else:
        output = ""
        for idx, char in enumerate(text):
            if char.isupper():
                converted_char = char.lower()
                if idx != 0 or len(char) -1:
                    converted_char = "_" + converted_char
                output += converted_char
            else:
                output += char
    
        return output


In [13]:
scores.columns = [normalize(col) for col in scores.columns]


BMI


In [14]:
patient_data = pd.read_csv("../dataset/scoring_data/patient_data.csv", index_col="PatientID")
patient_data.columns = [normalize(col) for col in patient_data.columns]

patient_data.head()

,birth_year,city,state,country
PatientID,,,,
18022,1992,Lewiston,Maine,United States
18024,1981,West Mifflin,Pennsylvania,United States
18051,1966,Saint George,Utah,United States
18114,1996,Grand Forks,North Dakota,United States
18147,1975,Rancho Palos Verdes,California,United States


In [15]:
combined = scores.merge(patient_data, how="left", on="PatientID")


In [16]:
combined.head()

,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree_function,outcome,birth_year,city,state,country
PatientID,,,,,,,,,,,
18022,129,80,0,0,31.2,0.703,0,1992,Lewiston,Maine,United States
18024,159,64,0,0,27.4,0.294,0,1981,West Mifflin,Pennsylvania,United States
18051,137,61,0,0,24.2,0.151,0,1966,Saint George,Utah,United States
18114,113,50,10,85,29.5,0.626,0,1996,Grand Forks,North Dakota,United States
18147,105,90,0,0,29.6,0.197,0,1975,Rancho Palos Verdes,California,United States


In [17]:
pregnancy_records = pd.read_csv("../dataset/scoring_data/pregnancies_records.csv", index_col="PatientID")

pregnancy_records.columns = [normalize(col) for col in pregnancy_records.columns]

In [18]:
output = []

for id in combined.index:
    if id in pregnancy_records.index:
        output.append(len(pregnancy_records.loc[id]))
    else:
        output.append(0)

combined["number_of_pregnancies"] = output

In [19]:
from datetime import datetime

# convert birth year to age

combined.birth_year.dtype

age = [datetime.now().year - year for year in combined.birth_year]

combined["age"] = age

In [21]:
combined["glucose_insulin_difference"] = np.absolute(scores["glucose"] - scores["insulin"])

In [23]:
combined.head()

,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree_function,outcome,birth_year,city,state,country,number_of_pregnancies,age,glucose_insulin_difference
PatientID,,,,,,,,,,,,,,
18022,129,80,0,0,31.2,0.703,0,1992,Lewiston,Maine,United States,0,32,129
18024,159,64,0,0,27.4,0.294,0,1981,West Mifflin,Pennsylvania,United States,7,43,159
18051,137,61,0,0,24.2,0.151,0,1966,Saint George,Utah,United States,6,58,137
18114,113,50,10,85,29.5,0.626,0,1996,Grand Forks,North Dakota,United States,3,28,28
18147,105,90,0,0,29.6,0.197,0,1975,Rancho Palos Verdes,California,United States,0,49,105


In [31]:
corrs = combined.corr(numeric_only=True)["outcome"]


In [32]:
X = combined[corrs.index]

X.to_csv("../feature_store/feature_data_milestone_3.csv")